# Download the libraries

In [ ]:
! rm -rf jars
!mkdir jars
!wget -q -P jars https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.4.1/spark-sql-kafka-0-10_2.12-3.4.1.jar
!wget -q -P jars https://repo1.maven.org/maven2/org/apache/kafka/kafka-clients/3.5.1/kafka-clients-3.5.1.jar
!wget -q -P jars https://repo1.maven.org/maven2/org/apache/spark/spark-token-provider-kafka-0-10_2.12/3.4.1/spark-token-provider-kafka-0-10_2.12-3.4.1.jar
!wget -q -P jars https://repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.18/scala-library-2.12.18.jar
!wget -q -P jars https://repo1.maven.org/maven2/org/apache/commons/commons-pool2/2.11.1/commons-pool2-2.11.1.jar

# Set up SparkSession

In [1]:
import os
from pyspark.sql import SparkSession

In [2]:
base_dir = os.getcwd() + '/jars'

spark = (SparkSession.builder
    .master('local[*]')
    .appName('Spark Structured Streaming example with Kafka')
    .config("spark.jars", 
            base_dir + '/kafka-clients-3.5.1.jar' + "," + 
            base_dir +'/spark-sql-kafka-0-10_2.12-3.4.1.jar' + "," + 
            base_dir + '/spark-token-provider-kafka-0-10_2.12-3.4.1.jar' + "," + 
            base_dir + '/scala-library-2.12.18.jar' + "," + 
            base_dir + '/commons-pool2-2.11.1.jar')
    .getOrCreate())

spark

# Define the schema for our data

In [250]:
from pyspark.sql.types import *

In [251]:
schema = StructType([
    StructField("VP", StructType([
      StructField("desi", StringType()),
      StructField("dir", StringType()),
      StructField("oper", IntegerType()),
      StructField("veh", IntegerType()),
      StructField("tst", TimestampType()),
      StructField("tsi", LongType()),
      StructField("spd", DoubleType()),
      StructField("hdg", IntegerType()),
      StructField("lat", DoubleType()),
      StructField("long", DoubleType()),
      StructField("acc", DoubleType()),
      StructField("dl", IntegerType()),
      StructField("odo", StringType()),
      StructField("drst", StringType()),
      StructField("oday", DateType()),
      StructField("jrn", IntegerType()),
      StructField("line", IntegerType()),
      StructField("start", StringType()),
      StructField("loc", StringType()),
      StructField("stop", LongType()),
      StructField("route", StringType()),
      StructField("occu", IntegerType())
    ]))
])

# Initialize the stream

In [252]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

We will read the data from the topic `vehicle-positions` in the Kafka cluster

In [253]:
kafka_source_df = (spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "broker:29092")
    .option("subscribe", "vehicle-positions")
    .option("startingOffsets", "earliest")
    .load()
    .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)"))

In [254]:
from pyspark.sql.functions import *


In [255]:
vehicle_position_df = (kafka_source_df
    .select(from_json(col("value"), schema).alias("json")) 
    .select("json.VP.*"))

In [256]:
vehicle_position_df.printSchema()

root
 |-- desi: string (nullable = true)
 |-- dir: string (nullable = true)
 |-- oper: integer (nullable = true)
 |-- veh: integer (nullable = true)
 |-- tst: timestamp (nullable = true)
 |-- tsi: long (nullable = true)
 |-- spd: double (nullable = true)
 |-- hdg: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- acc: double (nullable = true)
 |-- dl: integer (nullable = true)
 |-- odo: string (nullable = true)
 |-- drst: string (nullable = true)
 |-- oday: date (nullable = true)
 |-- jrn: integer (nullable = true)
 |-- line: integer (nullable = true)
 |-- start: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- stop: long (nullable = true)
 |-- route: string (nullable = true)
 |-- occu: integer (nullable = true)



<h3>Record Sample</h3> 

<code>
{
  "desi": "M1",
  "dir": "1",
  "oper": 50,
  "veh": 302,
  "tst": "2023-08-28T09:57:56Z",
  "tsi": 1693216676,
  "spd": 11.86,
  "hdg": 52,
  "lat": 60.1721918,
  "long": 24.94817722,
  "acc": null,
  "dl": null,
  "odo": null,
  "drst": null,
  "oday": "2023-08-28",
  "start": "12:26",
  "loc": "MAN",
  "stop": 1020603,
  "route": "31M1",
  "occu": 0,
  "seq": 1
}
</code>

### Perform streaming transformations

[window documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.window.html#pyspark.sql.functions.window)<br>
[withWatermark documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withWatermark.html#pyspark.sql.DataFrame.withWatermark)

In [257]:
vehicle_position_window_df = (vehicle_position_df
      .withWatermark("tst", "1 milliseconds")
      .groupBy(
        window(col("tst"), "1 minutes", "1 minutes"), #window(timeColumn, windowDuration, slideDuration) (tumbling windows: an input can belong only to one window)
        col("route")
      ).count())

In [258]:
vehicle_position_window_df.printSchema()

root
 |-- window: struct (nullable = false)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- route: string (nullable = true)
 |-- count: long (nullable = false)



# Start the streaming query

## Append mode

We are using the sink **memory**, it support only two output modes: **append** and **complete**.

In [259]:
query_append_trigger = (vehicle_position_window_df.writeStream
    .format("memory")
    .trigger(processingTime = '6 seconds') 
    .outputMode("append")
    .queryName("query_append_trigger")
    .start())

In [262]:
(spark.sql("select * from query_append_trigger")
    .filter(col('route') == '2113')
    .show(n = 100, truncate = False))

+------------------------------------------+-----+-----+
|window                                    |route|count|
+------------------------------------------+-----+-----+
|{2023-10-31 11:19:00, 2023-10-31 11:20:00}|2113 |5    |
|{2023-10-31 11:22:00, 2023-10-31 11:23:00}|2113 |6    |
|{2023-10-31 13:40:00, 2023-10-31 13:41:00}|2113 |60   |
|{2023-10-31 11:21:00, 2023-10-31 11:22:00}|2113 |180  |
|{2023-10-31 11:14:00, 2023-10-31 11:15:00}|2113 |134  |
|{2023-10-31 11:09:00, 2023-10-31 11:10:00}|2113 |50   |
|{2023-10-31 11:13:00, 2023-10-31 11:14:00}|2113 |7    |
|{2023-10-31 11:20:00, 2023-10-31 11:21:00}|2113 |180  |
|{2023-10-31 13:42:00, 2023-10-31 13:43:00}|2113 |147  |
|{2023-10-31 13:41:00, 2023-10-31 13:42:00}|2113 |120  |
+------------------------------------------+-----+-----+



In [263]:
query_append_trigger.stop()

## Complete mode

This Dataframe is identical to `vehicle_position_window_df`, but I have removed the .withWatermark because it is useless in complete mode

In [264]:
vehicle_position_window_no_wm_df = (vehicle_position_df
      .groupBy(
        window(col("tst"), "1 minutes", "1 minutes"), #window(timeColumn, windowDuration, slideDuration) (tumbling windows: an input can belong only to one window)
        col("route")
      ).count())

In [265]:
query_complete = (vehicle_position_window_no_wm_df.writeStream
            .format("memory")
            .trigger(processingTime = '6 seconds') 
            .outputMode("complete")
            .queryName("query_complete")
            .start())

In [268]:
(spark.sql("select * from query_complete")
    .filter(col('route') == '2113')
    .show(n = 100, truncate = False))

+------------------------------------------+-----+-----+
|window                                    |route|count|
+------------------------------------------+-----+-----+
|{2023-10-31 11:19:00, 2023-10-31 11:20:00}|2113 |5    |
|{2023-10-31 11:22:00, 2023-10-31 11:23:00}|2113 |6    |
|{2023-10-31 13:43:00, 2023-10-31 13:44:00}|2113 |84   |
|{2023-10-31 13:40:00, 2023-10-31 13:41:00}|2113 |60   |
|{2023-10-31 11:21:00, 2023-10-31 11:22:00}|2113 |180  |
|{2023-10-31 11:14:00, 2023-10-31 11:15:00}|2113 |134  |
|{2023-10-31 11:09:00, 2023-10-31 11:10:00}|2113 |50   |
|{2023-10-31 11:13:00, 2023-10-31 11:14:00}|2113 |7    |
|{2023-10-31 11:20:00, 2023-10-31 11:21:00}|2113 |180  |
|{2023-11-02 13:53:00, 2023-11-02 13:54:00}|2113 |44   |
|{2023-10-31 13:42:00, 2023-10-31 13:43:00}|2113 |147  |
|{2023-10-31 13:41:00, 2023-10-31 13:42:00}|2113 |120  |
+------------------------------------------+-----+-----+



In [269]:
query_complete.stop()

<h3> Thoughts on Complete mode </h3>
The complete mode never releases the intermediate state of the aggregation which means that the memory consumption will increase indefinitely for a query such this. 

### Thoughts on watermarking output mode and sink

- The watermarking can only be used in **update** and **append** mode. 
- In **complete** mode the old aggregation state is never dropped and we cannot use watermarking.
- Without the watermarking in the **append** mode the old aggregation state is never dropped.
- The sink format **memory** should be used only for debugging purposes and with low volumes of data since the output is entirely stored in the driver's memory
- The sink format **memory** does not support the **update** mode.

# Join Stream Stream

In [6]:
rate_source_df = (spark 
    .readStream 
    .format("rate")
    .load())

The input source `rate` generates one row per second with a timestamp and an increasing value:
<code>
+-----------------------+-----+
|timestamp              |value|
+-----------------------+-----+
|2023-10-31 14:16:43.267|0    |
|2023-10-31 14:16:44.267|1    |
|2023-10-31 14:16:45.267|2    |
|2023-10-31 14:16:46.267|3    |
|2023-10-31 14:16:47.267|4    |
|2023-10-31 14:16:51.267|8    |
|2023-10-31 14:16:48.267|5    |
|2023-10-31 14:16:52.267|9    |
|2023-10-31 14:16:49.267|6    |
|2023-10-31 14:16:50.267|7    |
+-----------------------+-----+
</code>

In [11]:
second_rate_source_df = (spark 
    .readStream 
    .format("rate")
    .load()
    .withColumn("animals", 
                when(col("value") % 4 == 0, lit("Cat"))
                .when(col("value") % 4 == 1, lit("Dog"))
                .when(col("value") % 4 == 2, lit("Mouse"))
                .otherwise(lit("Horse")))
    )

<code>
+-----------------------+-----+-------+
|timestamp              |value|animals|
+-----------------------+-----+-------+
|2023-11-02 09:51:49.535|0    |Cat    |
|2023-11-02 09:51:50.535|1    |Dog    |
|2023-11-02 09:51:51.535|2    |Mouse  |
|2023-11-02 09:51:52.535|3    |Horse  |
|2023-11-02 09:51:53.535|4    |Cat    |
|2023-11-02 09:51:57.535|8    |Cat    |
|2023-11-02 09:51:54.535|5    |Dog    |
|2023-11-02 09:51:58.535|9    |Dog    |
|2023-11-02 09:51:55.535|6    |Mouse  |
|2023-11-02 09:51:56.535|7    |Horse  |
+-----------------------+-----+-------+
</code>

To allow the state cleaning when performing a join stream-stream we need to specify two conditions:
- watermarking
- range condition on the event time

In [12]:
# Apply watermarks on event-time columns
rate_source_wm_df = (rate_source_df
    .withColumnRenamed("timestamp", "timestamp_rate_source")
    .withColumnRenamed("value", "value_rate_source")
    .withWatermark("timestamp_rate_source", "1 milliseconds"))

In [ ]:
query_rate_source_wm = (rate_source_wm_df.writeStream
    .format("memory")
    .trigger(processingTime = '6 seconds') 
    .outputMode("append")
    .queryName("query_rate_source_wm")
    .start())

In [ ]:
(spark.sql("select * from query_rate_source_wm")
    .show(n = 100, truncate = False))

In [ ]:
query_rate_source_wm.stop()

In [185]:
# Apply watermarks on event-time columns
second_rate_source_wm_df = (second_rate_source_df
    .withColumn("timestamp_second_source", col("timestamp") + expr("INTERVAL 2 seconds")) #add 5 seconds 
    .withColumnRenamed("value", "value_second_source")
    .drop("timestamp")                        
    .withWatermark("timestamp_second_source", "10 milliseconds"))

In [114]:
second_rate_source_wm = (second_rate_source_wm_df.writeStream
    .format("memory")
    .trigger(processingTime = '6 seconds') 
    .outputMode("append")
    .queryName("second_rate_source_wm")
    .start())

In [102]:
(spark.sql("select * from second_rate_source_wm")
    .show(n = 100, truncate = False))

+-----------------------+-------------------+-------+-----------------------------+
|timestamp_second_source|value_second_source|animals|timestamp_second_source_plus5|
+-----------------------+-------------------+-------+-----------------------------+
|2023-11-02 11:53:13.108|0                  |Cat    |2023-11-02 11:53:33.108      |
|2023-11-02 11:53:14.108|1                  |Dog    |2023-11-02 11:53:34.108      |
|2023-11-02 11:53:15.108|2                  |Mouse  |2023-11-02 11:53:35.108      |
|2023-11-02 11:53:16.108|3                  |Horse  |2023-11-02 11:53:36.108      |
+-----------------------+-------------------+-------+-----------------------------+



In [115]:
second_rate_source_wm.stop()

## Inner Join

In [214]:
# Join with event-time constraints
join_rate_sources_wm_df = second_rate_source_wm_df.join(
  rate_source_wm_df, 
  expr("""
    value_rate_source = value_second_source AND
    timestamp_second_source >= timestamp_rate_source AND
    timestamp_second_source <= timestamp_rate_source + interval 3 seconds
    """)
)

In [215]:
join_rate_sources_wm_query = (join_rate_sources_wm_df.writeStream
    .format("memory")
    .trigger(processingTime = '6 seconds') 
    .outputMode("append")
    .queryName("join_rate_sources_wm_query")
    .start())

In [218]:
(spark.sql("select * from join_rate_sources_wm_query")
    .show(n = 100, truncate = False))

+-------------------+-------+-----------------------+-----------------------+-----------------+
|value_second_source|animals|timestamp_second_source|timestamp_rate_source  |value_rate_source|
+-------------------+-------+-----------------------+-----------------------+-----------------+
|0                  |Cat    |2023-11-02 13:35:11.268|2023-11-02 13:35:09.295|0                |
|6                  |Mouse  |2023-11-02 13:35:17.268|2023-11-02 13:35:15.295|6                |
|5                  |Dog    |2023-11-02 13:35:16.268|2023-11-02 13:35:14.295|5                |
|1                  |Dog    |2023-11-02 13:35:12.268|2023-11-02 13:35:10.295|1                |
|3                  |Horse  |2023-11-02 13:35:14.268|2023-11-02 13:35:12.295|3                |
|2                  |Mouse  |2023-11-02 13:35:13.268|2023-11-02 13:35:11.295|2                |
|4                  |Cat    |2023-11-02 13:35:15.268|2023-11-02 13:35:13.295|4                |
+-------------------+-------+-----------

In [219]:
join_rate_sources_wm_query.stop()

In [ ]:
join_rate_sources_wm_query.exception()

In [ ]:
join_rate_sources_wm_query.lastProgress

## Left outer join

In [237]:
rate_source_wm_filter_df = rate_source_wm_df.filter(col("value_rate_source") % 3 == 0 )

In [238]:
# Join with event-time constraints
join_rate_sources_wm_filter_df = second_rate_source_wm_df.join(
  rate_source_wm_filter_df, 
  expr("""
    value_rate_source = value_second_source AND
    timestamp_second_source >= timestamp_rate_source AND
    timestamp_second_source <= timestamp_rate_source + interval 3 seconds
    """),
    "leftOuter"
)

In [239]:
join_rate_sources_wm_filter_query = (join_rate_sources_wm_filter_df.writeStream
    .format("memory")
    .trigger(processingTime = '6 seconds') 
    .outputMode("append")
    .queryName("join_rate_sources_wm_filter_query")
    .start())

In [244]:
(spark.sql("select * from join_rate_sources_wm_filter_query")
    .show(n = 100, truncate = False))

+-------------------+-------+-----------------------+-----------------------+-----------------+
|value_second_source|animals|timestamp_second_source|timestamp_rate_source  |value_rate_source|
+-------------------+-------+-----------------------+-----------------------+-----------------+
|0                  |Cat    |2023-11-02 13:50:28.437|2023-11-02 13:50:26.471|0                |
|6                  |Mouse  |2023-11-02 13:50:34.437|2023-11-02 13:50:32.471|6                |
|3                  |Horse  |2023-11-02 13:50:31.437|2023-11-02 13:50:29.471|3                |
|9                  |Dog    |2023-11-02 13:50:37.437|2023-11-02 13:50:35.471|9                |
|1                  |Dog    |2023-11-02 13:50:29.437|null                   |null             |
|12                 |Cat    |2023-11-02 13:50:40.437|2023-11-02 13:50:38.471|12               |
|2                  |Mouse  |2023-11-02 13:50:30.437|null                   |null             |
|4                  |Cat    |2023-11-02 

In [245]:
join_rate_sources_wm_filter_query.stop()

# Join Stream-Static

In [270]:
operators_df = spark.read.csv("operators.csv", header = True, inferSchema = True)

In [271]:
operators_df.show(truncate = False)

+---+------------------------------------+-------------+----------------------+-----------------------+
|id |operator_name                       |country      |city                  |address                |
+---+------------------------------------+-------------+----------------------+-----------------------+
|1  |Considine LLC                       |France       |Soisy-sous-Montmorency|4 Nelson Park          |
|2  |Mraz LLC                            |Canada       |Rayside-Balfour       |36 Hallows Way         |
|3  |Hudson, Stehr and Satterfield       |Japan        |Hirara                |78 Westport Trail      |
|4  |Grant-Gorczany                      |Nepal        |Dadeldhurā            |739 Glacier Hill Avenue|
|5  |Abshire Inc                         |Peru         |Pilpichaca            |05773 Veith Pass       |
|6  |Bins-Schroeder                      |Thailand     |Nong Don              |5 Washington Place     |
|7  |Muller, Glover and Daugherty        |Ukraine      |Chynadiy

In [272]:
operators_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- operator_name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- address: string (nullable = true)



In [273]:
join_vehicle_operators_df = vehicle_position_df.join(operators_df, vehicle_position_df.oper == operators_df.id, "left_outer")

In [274]:
join_vehicle_operators_df.printSchema()

root
 |-- desi: string (nullable = true)
 |-- dir: string (nullable = true)
 |-- oper: integer (nullable = true)
 |-- veh: integer (nullable = true)
 |-- tst: timestamp (nullable = true)
 |-- tsi: long (nullable = true)
 |-- spd: double (nullable = true)
 |-- hdg: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- acc: double (nullable = true)
 |-- dl: integer (nullable = true)
 |-- odo: string (nullable = true)
 |-- drst: string (nullable = true)
 |-- oday: date (nullable = true)
 |-- jrn: integer (nullable = true)
 |-- line: integer (nullable = true)
 |-- start: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- stop: long (nullable = true)
 |-- route: string (nullable = true)
 |-- occu: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- operator_name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- address: string (nullable = true)



In [275]:
query_join_vechicle_operators_append = (join_vehicle_operators_df.writeStream
    .format("memory")
    .trigger(processingTime = '6 seconds') 
    .outputMode("append")
    .queryName("query_join_vechicle_operators_append")
    .start())

In [277]:
(spark.sql("select tst, route, oper, operator_name, country, city, address from query_join_vechicle_operators_append")
    .filter(col('route') == '2113')
    .show(n = 20, truncate = False))

+-----------------------+-----+----+---------------+-------+----------+------------------+
|tst                    |route|oper|operator_name  |country|city      |address           |
+-----------------------+-----+----+---------------+-------+----------+------------------+
|2023-10-31 11:09:04.481|2113 |12  |Pollich-Kuhlman|Sudan  |Al Manāqil|7 Golf View Circle|
|2023-10-31 11:09:05.481|2113 |12  |Pollich-Kuhlman|Sudan  |Al Manāqil|7 Golf View Circle|
|2023-10-31 11:09:18.509|2113 |12  |Pollich-Kuhlman|Sudan  |Al Manāqil|7 Golf View Circle|
|2023-10-31 11:09:19.509|2113 |12  |Pollich-Kuhlman|Sudan  |Al Manāqil|7 Golf View Circle|
|2023-10-31 11:09:20.51 |2113 |12  |Pollich-Kuhlman|Sudan  |Al Manāqil|7 Golf View Circle|
|2023-10-31 11:09:21.509|2113 |12  |Pollich-Kuhlman|Sudan  |Al Manāqil|7 Golf View Circle|
|2023-10-31 11:09:22.509|2113 |12  |Pollich-Kuhlman|Sudan  |Al Manāqil|7 Golf View Circle|
|2023-10-31 11:09:23.481|2113 |12  |Pollich-Kuhlman|Sudan  |Al Manāqil|7 Golf View Circle|

In [278]:
query_join_vechicle_operators_append.stop()

## Aggregations after join

In [279]:
vehicle_position_operators_group_df = (join_vehicle_operators_df
      .withWatermark("tst", "1 milliseconds")
      .groupBy(
        window(col("tst"), "1 minutes", "1 minutes"),
        col("oper"),
        col("operator_name") 
      ).count())

In [280]:
vehicle_position_operators_query_append = (vehicle_position_operators_group_df.writeStream
        .format("memory")
        .trigger(processingTime = '6 seconds') 
        .outputMode("append")
        .queryName("vehicle_position_operators_append")
        .start())

In [283]:
(spark.sql("select * from vehicle_position_operators_append")
    .filter(col('oper') == '22')
    .show(truncate = False, n = 30))

+------------------------------------------+----+------------------------------+-----+
|window                                    |oper|operator_name                 |count|
+------------------------------------------+----+------------------------------+-----+
|{2023-10-31 11:08:00, 2023-10-31 11:09:00}|22  |Mueller, Hoeger and Morissette|2    |
|{2023-10-31 11:20:00, 2023-10-31 11:21:00}|22  |Mueller, Hoeger and Morissette|15622|
|{2023-10-31 11:13:00, 2023-10-31 11:14:00}|22  |Mueller, Hoeger and Morissette|564  |
|{2023-11-02 13:52:00, 2023-11-02 13:53:00}|22  |Mueller, Hoeger and Morissette|28   |
|{2023-10-31 13:43:00, 2023-10-31 13:44:00}|22  |Mueller, Hoeger and Morissette|9618 |
|{2023-10-31 11:22:00, 2023-10-31 11:23:00}|22  |Mueller, Hoeger and Morissette|596  |
|{2023-10-31 13:42:00, 2023-10-31 13:43:00}|22  |Mueller, Hoeger and Morissette|20599|
|{2023-10-31 13:41:00, 2023-10-31 13:42:00}|22  |Mueller, Hoeger and Morissette|20336|
|{2023-10-31 11:19:00, 2023-10-31 11:20:00}

In [284]:
vehicle_position_operators_query_append.stop()

## Sorting

In [285]:
vehicle_position_operators_group_sort_df = (join_vehicle_operators_df
      .groupBy(
        window(col("tst"), "1 minutes", "1 minutes"),
        col("oper"),
        col("operator_name") 
      ).count()
      .orderBy(desc('window')))

In [286]:
vehicle_position_operators_query_complete = (vehicle_position_operators_group_sort_df.writeStream
    .format("memory")
    .trigger(processingTime = '6 seconds') 
    .outputMode("complete")
    .queryName("vehicle_position_operators_complete")
    .start())

In [287]:
(spark.sql("select * from vehicle_position_operators_complete")
    .filter(col('oper') == '22')
    .show(truncate = False, n = 100))

+------------------------------------------+----+------------------------------+-----+
|window                                    |oper|operator_name                 |count|
+------------------------------------------+----+------------------------------+-----+
|{2023-11-02 13:53:00, 2023-11-02 13:54:00}|22  |Mueller, Hoeger and Morissette|5223 |
|{2023-11-02 13:52:00, 2023-11-02 13:53:00}|22  |Mueller, Hoeger and Morissette|28   |
|{2023-10-31 13:43:00, 2023-10-31 13:44:00}|22  |Mueller, Hoeger and Morissette|9618 |
|{2023-10-31 13:42:00, 2023-10-31 13:43:00}|22  |Mueller, Hoeger and Morissette|20599|
|{2023-10-31 13:41:00, 2023-10-31 13:42:00}|22  |Mueller, Hoeger and Morissette|20336|
|{2023-10-31 13:40:00, 2023-10-31 13:41:00}|22  |Mueller, Hoeger and Morissette|10223|
|{2023-10-31 11:22:00, 2023-10-31 11:23:00}|22  |Mueller, Hoeger and Morissette|596  |
|{2023-10-31 11:21:00, 2023-10-31 11:22:00}|22  |Mueller, Hoeger and Morissette|15776|
|{2023-10-31 11:20:00, 2023-10-31 11:21:00}

In [ ]:
vehicle_position_operators_query_complete.stop()

<h3> Thoughts on sorting </h3>
<b>Sorting operations are supported on streaming Datasets only after an aggregation and in Complete Output Mode</b>. <br>
So we can only use the groupBy in Complete mode and not in Append mode.

<h1>Query mamagement</h1>

[documentation](https://spark.apache.org/docs/3.4.1/api/python/reference/pyspark.ss/api/pyspark.sql.streaming.StreamingQuery.html)

<h3>Get the unique identifier of the running query</h3>

In [288]:
vehicle_position_operators_query_complete.id

'718b7907-b6a0-47bd-9c91-a07f39297a25'

<h3>Get the run id of the query</h3>

In [289]:
vehicle_position_operators_query_complete.runId

'e45cc28f-359d-4a56-a213-0c7e2d9a1465'

<h3>Get the name of the auto-generated or user-specified name</h3>

In [290]:
vehicle_position_operators_query_complete.name

'vehicle_position_operators_complete'

<h3>Print detailed explanations of the query</h3>

In [291]:
vehicle_position_operators_query_complete.explain()

== Physical Plan ==
WriteToDataSourceV2 MicroBatchWrite[epoch: 0, writer: org.apache.spark.sql.execution.streaming.sources.MemoryStreamingWrite@62a7e55c], org.apache.spark.sql.execution.datasources.v2.DataSourceV2Strategy$$Lambda$2247/0x0000000801cceb58@6094b0c3
+- *(6) Sort [window#1002424 DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(window#1002424 DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=69893]
      +- *(5) HashAggregate(keys=[window#1002424, oper#984008, operator_name#996269], functions=[count(1)])
         +- StateStoreSave [window#1002424, oper#984008, operator_name#996269], state info [ checkpoint = file:/tmp/temporary-d6a1d74d-cd79-4991-a64e-9814a8fc0dc3/state, runId = e45cc28f-359d-4a56-a213-0c7e2d9a1465, opId = 0, ver = 0, numPartitions = 200], Complete, 0, 0, 2
            +- *(4) HashAggregate(keys=[window#1002424, oper#984008, operator_name#996269], functions=[merge_count(1)])
               +- StateStoreRestore [window#1002424, oper#984008, opera

<h3>Query recent progress</h3>

In [292]:
vehicle_position_operators_query_complete.recentProgress

[{'id': '718b7907-b6a0-47bd-9c91-a07f39297a25',
  'runId': 'e45cc28f-359d-4a56-a213-0c7e2d9a1465',
  'name': 'vehicle_position_operators_complete',
  'timestamp': '2023-11-02T13:55:24.645Z',
  'batchId': 0,
  'numInputRows': 411015,
  'inputRowsPerSecond': 0.0,
  'processedRowsPerSecond': 48698.459715639816,
  'durationMs': {'addBatch': 8331,
   'commitOffsets': 23,
   'getBatch': 1,
   'latestOffset': 38,
   'queryPlanning': 23,
   'triggerExecution': 8440,
   'walCommit': 23},
  'stateOperators': [{'operatorName': 'stateStoreSave',
    'numRowsTotal': 414,
    'numRowsUpdated': 414,
    'allUpdatesTimeMs': 2159,
    'numRowsRemoved': 0,
    'allRemovalsTimeMs': 0,
    'commitTimeMs': 12985,
    'memoryUsedBytes': 208768,
    'numRowsDroppedByWatermark': 0,
    'numShufflePartitions': 400,
    'numStateStoreInstances': 400,
    'customMetrics': {'loadedMapCacheHitCount': 0,
     'loadedMapCacheMissCount': 0,
     'stateOnCurrentVersionSizeBytes': 151168}}],
  'sources': [{'description

<h3>Get the query last progress</h3>

In [293]:
vehicle_position_operators_query_complete.lastProgress

{'id': '718b7907-b6a0-47bd-9c91-a07f39297a25',
 'runId': 'e45cc28f-359d-4a56-a213-0c7e2d9a1465',
 'name': 'vehicle_position_operators_complete',
 'timestamp': '2023-11-02T13:55:48.000Z',
 'batchId': 1,
 'numInputRows': 0,
 'inputRowsPerSecond': 0.0,
 'processedRowsPerSecond': 0.0,
 'durationMs': {'latestOffset': 3, 'triggerExecution': 3},
 'stateOperators': [{'operatorName': 'stateStoreSave',
   'numRowsTotal': 414,
   'numRowsUpdated': 0,
   'allUpdatesTimeMs': 2159,
   'numRowsRemoved': 0,
   'allRemovalsTimeMs': 0,
   'commitTimeMs': 12985,
   'memoryUsedBytes': 208768,
   'numRowsDroppedByWatermark': 0,
   'numShufflePartitions': 400,
   'numStateStoreInstances': 400,
   'customMetrics': {'loadedMapCacheHitCount': 0,
    'loadedMapCacheMissCount': 0,
    'stateOnCurrentVersionSizeBytes': 151168}}],
 'sources': [{'description': 'KafkaV2[Subscribe[vehicle-positions]]',
   'startOffset': {'vehicle-positions': {'2': 69199,
     '5': 69539,
     '4': 68791,
     '1': 65759,
     '3': 69

<h3>Get the list of currently active streaming queries</h3>

In [294]:
spark.streams.active

<h3>Get the query object from the id</h3>

In [295]:
query = spark.streams.get(vehicle_position_operators_query_complete.id)

In [296]:
query.name

'vehicle_position_operators_complete'

<h3>Query active</h3>

In [297]:
vehicle_position_operators_query_complete.isActive

True

<h3>Query status</h3>

In [298]:
vehicle_position_operators_query_complete.status

{'message': 'Waiting for next trigger',
 'isDataAvailable': False,
 'isTriggerActive': False}

<h3>Query exception (useful if the query has terminated with an exception)</h3>

In [299]:
vehicle_position_operators_query_complete.exception()

<h3>Await query termination</h3>

In [300]:
#query.awaitTermination([timeout])

<h3>Stop the query</h3>

In [301]:
vehicle_position_operators_query_complete.stop()

<h1>Output Sinks</h1>

<h2>CSV</h2>

In [302]:
vehicle_position_operators_filter_df = (join_vehicle_operators_df
      .withWatermark("tst", "1 seconds")
      .filter(col('oper').isin('22','90','6','30','12','50'))                                  
      .groupBy(
        window(col("tst"), "1 minutes", "1 minutes"),
        col("oper"),
        col("operator_name") 
      ).count())

In [303]:
vehicle_position_operators_filter_mod_df = (vehicle_position_operators_filter_df 
        .withColumn("window",vehicle_position_operators_filter_df.window.cast('string'))
        .coalesce(1)) # writes 1 file csv for each trigger

In [305]:
vehicle_position_operators_filter_mod = (vehicle_position_operators_filter_mod_df.writeStream
        .format("csv")                               # can be "orc", "json", "parquet", etc.
        .option("path", "csv")
        .option("header",True)
        .trigger(processingTime = '1 minutes') 
        .outputMode("append")
        .option("checkpointLocation", "checkpoint")
        .start())

In [306]:
vehicle_position_operators_filter_mod.exception()

In [307]:
vehicle_position_operators_filter_mod.status

{'message': 'Waiting for next trigger',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [308]:
vehicle_position_operators_filter_mod.isActive

True

In [309]:
vehicle_position_operators_filter_mod.stop()

In [ ]:
# to delete folders
#! rm -rf csv

<h3>Thoughts on CSV sink</h3>
<ul>
    <li> It is not allowed to use the Complete mode so we cannot perform any sorting on the data. </li>
    <li> It is necessary to cast the column window to string because timestamp columns are not allowed when the sink is of type CSV </li>
    <li> It will be generated one CSV file for each trigger </li>
</ul>

<h2>Kafka Topic</h2>

In [310]:
vehicle_position_operators_kafka_df = (join_vehicle_operators_df
      .withWatermark("tst", "1 seconds")
      #.filter(col('oper').isin('22','90','6','30','12','50'))                                  
      .groupBy(
        window(col("tst"), "1 minutes", "1 minutes"),
        col("oper"),
        col("operator_name") 
      ).count()
      .select(col("oper").alias("key"), to_json(struct("window", "count", "operator_name")).alias("value")))  

In [311]:
vehicle_position_operators_kafka_df.printSchema()

root
 |-- key: integer (nullable = true)
 |-- value: string (nullable = true)



In [320]:
# Just to check the output
vehicle_position_operators_kafka_query_memory = (vehicle_position_operators_kafka_df
    .writeStream
    .format("memory")
    .trigger(processingTime = '1 minutes') 
    .outputMode("append")
    .queryName("vehicle_position_operators_kafka_query_memory")
    .start())

IllegalArgumentException: Cannot start query with name vehicle_position_operators_kafka_query_memory as a query with that name is already active in this SparkSession

In [321]:
(spark.sql("select * from vehicle_position_operators_kafka_query_memory")
    #.filter(col('key') == 22)
    .show(truncate = False, n = 100))

+---+-------------------------------------------------------------------------------------------------------------------------------------------------+
|key|value                                                                                                                                            |
+---+-------------------------------------------------------------------------------------------------------------------------------------------------+
|47 |{"window":{"start":"2023-10-31T13:41:00.000Z","end":"2023-10-31T13:42:00.000Z"},"count":1140,"operator_name":"Gerhold-Schultz"}                  |
|22 |{"window":{"start":"2023-10-31T11:08:00.000Z","end":"2023-10-31T11:09:00.000Z"},"count":2,"operator_name":"Mueller, Hoeger and Morissette"}      |
|60 |{"window":{"start":"2023-10-31T11:09:00.000Z","end":"2023-10-31T11:10:00.000Z"},"count":9,"operator_name":"Heathcote-Schinner"}                  |
|54 |{"window":{"start":"2023-10-31T11:19:00.000Z","end":"2023-10-31T11:20:00.000Z"},"co

In [ ]:
vehicle_position_operators_kafka_query_memory.stop()

Before continuing we must create the topic `operators-counts` by executing this line on a terminal: 
<br>
<br>
`docker exec -it broker kafka-topics --create --bootstrap-server broker:9092 --partitions 1 --replication-factor 1 --topic operators-counts`

In [ ]:
vehicle_position_operators_kafka_query_kafka = (vehicle_position_operators_kafka_df 
      .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") 
      .writeStream 
      .format("kafka") 
      .option("kafka.bootstrap.servers", "broker:29092") 
      .option("topic", "operators-counts")
      .option("checkpointLocation", "checkpoint-kafka")
      .start())

In [ ]:
vehicle_position_operators_kafka_query_kafka.status

To check if the data are being written to the topic `operators-count` in the Kafka cluster we can start a simple consumer by executing on a terminal:
<br>
<br>
`docker exec -it broker kafka-console-consumer --bootstrap-server broker:9092 --from-beginning --topic operators-counts --property print.key=true`

If everything went fine you should see something like this:
<br>
<br>
<code>
130	{"window":{"start":"2023-08-30T10:15:00.000Z","end":"2023-08-30T10:16:00.000Z"},"count":60,"operator_name":"Klein, Jakubowski and Hermiston"}
60	{"window":{"start":"2023-08-30T10:28:00.000Z","end":"2023-08-30T10:29:00.000Z"},"count":13,"operator_name":"Heathcote-Schinner"}
50	{"window":{"start":"2023-08-30T10:24:00.000Z","end":"2023-08-30T10:25:00.000Z"},"count":2062,"operator_name":"Lemke-Waters"}
30	{"window":{"start":"2023-08-30T10:19:00.000Z","end":"2023-08-30T10:20:00.000Z"},"count":1380,"operator_name":"Lakin, Breitenberg and Morissette"}
59	{"window":{"start":"2023-08-30T10:23:00.000Z","end":"2023-08-30T10:24:00.000Z"},"count":240,"operator_name":"Schuster Group"}
12	{"window":{"start":"2023-08-30T10:25:00.000Z","end":"2023-08-30T10:26:00.000Z"},"count":10595,"operator_name":"Pollich-Kuhlman"}
40	{"window":{"start":"2023-08-30T10:16:00.000Z","end":"2023-08-30T10:17:00.000Z"},"count":4432,"operator_name":"Bogisich LLC"}
54	{"window":{"start":"2023-08-30T10:29:00.000Z","end":"2023-08-30T10:30:00.000Z"},"count":540,"operator_name":"Morar Inc"}
21	{"window":{"start":"2023-08-30T10:27:00.000Z","end":"2023-08-30T10:28:00.000Z"},"count":240,"operator_name":"Ledner-Boyer"}
60	{"window":{"start":"2023-08-30T10:20:00.000Z","end":"2023-08-30T10:21:00.000Z"},"count":6,"operator_name":"Heathcote-Schinner"}
</code>


In [ ]:
vehicle_position_operators_kafka_query_kafka.stop()